This notebook is intended to help an analyst do some math checks on targets vs trajectories vs benchmarks for a company in a region and a sector.

It uses the OECM benchmark for production and intensity values. Subject data is formatted according to the rules of the ITR Data template.

**Cell 9 must be set manually** (based on sector/region of subject company set in **cell 8**)

In [ ]:
import os
import sys
import json
import argparse
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go 

In [ ]:
import ITR
from ITR.data.base_providers import BaseProviderProductionBenchmark, BaseProviderIntensityBenchmark
from ITR.interfaces import ETimeFrames, EScope, IProductionBenchmarkScopes, IEIBenchmarkScopes
from ITR.data.template import TemplateProviderCompany
from ITR.data.data_warehouse import DataWarehouse

In [ ]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

In [ ]:
self_root = os.path.abspath('')
benchmark_prod_json = os.path.join(self_root, "data", "json-units", "benchmark_production_OECM.json")
benchmark_EI_OECM = os.path.join(self_root, "data", "json-units", "benchmark_EI_OECM_S3.json")
benchmark_EI_TPI = os.path.join(self_root, "data", "json-units", "benchmark_EI_TPI_1_5_degrees.json")

# load production benchmarks
with open(benchmark_prod_json) as json_file:
    parsed_json = json.load(json_file)
prod_bms = IProductionBenchmarkScopes.parse_obj(parsed_json)
production_bm = BaseProviderProductionBenchmark(production_benchmarks=prod_bms)

# load intensity benchmarks

# OECM
with open(benchmark_EI_OECM) as json_file:
    parsed_json = json.load(json_file)
ei_bms = IEIBenchmarkScopes.parse_obj(parsed_json)
OECM_EI_bm = BaseProviderIntensityBenchmark(EI_benchmarks=ei_bms)

# TPI
with open(benchmark_EI_TPI) as json_file:
    parsed_json = json.load(json_file)
ei_bms = IEIBenchmarkScopes.parse_obj(parsed_json)
TPI_EI_bm = BaseProviderIntensityBenchmark(EI_benchmarks=ei_bms)

intensity_bm = TPI_EI_bm

In [ ]:
template_data_path = "data/20220927 ITR V2 Sample Data.xlsx"
# template_data_path = "data/20220927 ITR Tool Sample Data.xlsx"

# Remove the # and space on the next line to point the template_data_path variable at your own data
# template_data_path = "data/your_template_here.xlsx"

template_company_data = TemplateProviderCompany(excel_path=template_data_path)

In [ ]:
template_provider = DataWarehouse(template_company_data, production_bm, intensity_bm)

# Fills in template_company_data._companies[0].projected_targets.S1S2

print(f"Benchmark Temperature = {intensity_bm.benchmark_temperature}\n\
Benchmark Global Budget = {intensity_bm.benchmark_global_budget}\n\
AFOLU included = {intensity_bm.is_AFOLU_included}")

In [ ]:
data, idx = zip(*[(i, (bm.sector, bm.region))
                  for i, bm in enumerate(production_bm._productions_benchmarks.AnyScope.benchmarks)])
production_bm_mapper = pd.Series(data, idx)

if intensity_bm == OECM_EI_bm:
    # The OECM benchmarks are all coordinated with each other
    ei_s1_bm_mapper = None
    ei_s1s2_bm_mapper = ei_s1s2s3_bm_mapper = production_bm_mapper
else:
    # TPI benchmarks (possibly others) have their own rules
    data, idx = zip(*[(i, (bm.sector, bm.region))
                      for i, bm in enumerate(intensity_bm._EI_benchmarks.S1.benchmarks)])
    ei_s1_bm_mapper = pd.Series(data, idx)

    data, idx = zip(*[(i, (bm.sector, bm.region))
                      for i, bm in enumerate(intensity_bm._EI_benchmarks.S1S2.benchmarks)])
    ei_s1s2_bm_mapper = pd.Series(data, idx)

    data, idx = zip(*[(i, (bm.sector, bm.region))
                      for i, bm in enumerate(intensity_bm._EI_benchmarks.S1S2S3.benchmarks)])
    ei_s1s2s3_bm_mapper = pd.Series(data, idx)

In [ ]:
# US Steel US9129091081
# Carpenter Technologies US1442851036
# Cleveland Cliffs US1858991011
# Commercial Metals US2017231034
# Nucore US6703461052
# Steel Dynamics US8581191009
# Timken Steel US8873991033
# Worthington Industries US9818111026
# POSCO KR7005490008

company_ids = [ # 'GB0007980591', 'US1667641005', 'US30231G1022', 'US56585A1025', '2222.SR',
               'US9129091081', 'US1442851036', 'US1858991011', 'US2017231034', 'US6703461052',
               'US8581191009', 'US8873991033', 'US9818111026', 'KR7005490008']

models = template_provider.get_preprocessed_company_data(company_ids)
region = models[0].region if models[0].region in ['North America', 'Europe'] else 'Global'
sector_region_idx = production_bm_mapper.loc[models[0].sector, region]
sector_prod_baseline = production_bm._productions_benchmarks.AnyScope.benchmarks[sector_region_idx].base_year_production.m
print(f"set sector_prod_baseline (total units of output) for {models[0].sector} {region}; base_proudction_year = {sector_prod_baseline}")

In [ ]:
def get_ei_scope_by_sector_region(scope, sector, region, bm_mapper):
    """
    """
    if getattr(intensity_bm._EI_benchmarks, scope) and sector in bm_mapper.index:
        if (sector, region) in bm_mapper.index:
            ei_sector_region_idx = bm_mapper.loc[sector, region]
        elif (model.sector, "Global") in bm_mapper.index:
            ei_sector_region_idx = bm_mapper.loc[sector, "Global"]
        ei_data, ei_idx = zip(*[(ei.value, ei.year)
                              for ei in getattr(intensity_bm._EI_benchmarks, scope).benchmarks[ei_sector_region_idx].projections_nounits])
        sector_ei = pd.Series(ei_data, ei_idx)
    else:
        sector_ei = None
    return sector_ei

def get_em_projections(model_ei) -> pd.Series:
    """
    """
    if getattr(model_ei, 'S1S2S3'):
        data, idx = zip(*[(p.value.m, p.year) for p in model_ei.S1S2S3.projections])
    elif getattr(model_ei, 'S1S2'):
        data, idx = zip(*[(p.value.m, p.year) for p in model_ei.S1S2.projections])
    elif getattr(model_ei, 'S1'):
        data, idx = zip(*[(p.value.m, p.year) for p in model_ei.S1.projections])
    else:
        raise ValueError("no valid scope found for {model_ei}")
    return pd.Series(data, idx)

In [ ]:
sector_dfs = []

for model in models:
    region = model.region if model.region in ['North America', 'Europe'] else 'Global'
    sector_region_idx = production_bm_mapper.loc[model.sector, region]
    sector_prod_baseline = production_bm._productions_benchmarks.AnyScope.benchmarks[sector_region_idx].base_year_production.m
    prod_data, prod_idx = zip(*[(p.value, p.year)
                              for p in production_bm._productions_benchmarks.AnyScope.benchmarks[sector_region_idx].projections_nounits])
    sector_production = pd.Series(prod_data, prod_idx)

    if ei_s1_bm_mapper is not None:
        sector_ei_s1 = get_ei_scope_by_sector_region('S1', model.sector, region, ei_s1_bm_mapper)
    else:
        sector_ei_s1 = None
    sector_ei_s1s2 = get_ei_scope_by_sector_region('S1S2', model.sector, region, ei_s1s2_bm_mapper)
    sector_ei_s1s2s3 = get_ei_scope_by_sector_region('S1S2S3', model.sector, region, ei_s1s2s3_bm_mapper)
    sector_growth_partial = sector_production.add(1).cumprod()
    data, idx = zip(*[(p.value.m, p.year)
                  for p in model.historic_data.productions if p.year in [2019,2020] ])
    co_historic_productions = pd.Series(data, idx)

    co_projected_productions = (
        co_historic_productions[2020]
        * sector_growth_partial[sector_growth_partial.index>2020]
    )

    co_productions = pd.concat([co_historic_productions, co_projected_productions])

    co_ei_trajectory = get_em_projections(model.projected_intensities)
    co_ei_target = pd.concat([co_ei_trajectory.iloc[0:2],
                              get_em_projections(model.projected_targets)])
    plot_dict = {
        "Trajectory": (co_productions * co_ei_trajectory).cumsum(),
        "Target": (co_productions * co_ei_target).cumsum(),
    }
    if sector_ei_s1 is not None:
        plot_dict["BenchmarkS1"] = (co_productions[2019] * sector_growth_partial * sector_ei_s1s2).cumsum()
    if sector_ei_s1s2 is not None:
        plot_dict["BenchmarkS1S2"] = (co_productions[2019] * sector_growth_partial * sector_ei_s1s2).cumsum()
    if sector_ei_s1s2s3 is not None:
        plot_dict["BenchmarkS1S2S3"] = (co_productions[2019] * sector_growth_partial * sector_ei_s1s2s3).cumsum()
    sector_df = pd.DataFrame(plot_dict)
    display(px.line(sector_df, y=[k for k in plot_dict.keys()],
            labels={'index':'Year', 'value':'t CO2', 'variable':model.company_id}))